In [13]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


In [15]:
features_df = pd.read_csv('train_x.csv')
labels_df = pd.read_csv('train_y.csv')

In [17]:
features_df.drop(columns=['Unnamed: 0'], inplace=True)
labels_df.drop(columns=['Unnamed: 0'], inplace=True)

mask = (labels_df['year'] == 2000) | (labels_df['year'] == 2002)
X_features = features_df[mask].to_numpy()
y_labels = labels_df[mask].replace({2000: 0, 2002: 1}).to_numpy().ravel()

scaler = StandardScaler()
X_normalized = scaler.fit_transform(X_features)

In [19]:
def sigmoid(z):
    return 1.0 / (1.0 + torch.exp(-z))

In [21]:
def compute_nll(predictions, targets):
    epsilon = 1e-10 
    predictions = torch.clamp(predictions, epsilon, 1 - epsilon)
    return - (targets * torch.log(predictions) + (1 - targets) * torch.log(1 - predictions))

In [23]:
def train_sigmoid_neuron(X, y, weights, bias, learning_rate, num_epochs):
    X_tensor = torch.tensor(X, dtype=torch.float64)
    y_tensor = torch.tensor(y, dtype=torch.float64)    
    weight_tensor = torch.tensor(weights, dtype=torch.float64)
    bias_tensor = torch.tensor(bias, dtype=torch.float64)
    loss_history = []
    for epoch in range(num_epochs):
        cumulative_loss = 0.0
        total_weight_gradient = torch.zeros_like(weight_tensor)
        total_bias_gradient = 0.0       
        for i in range(len(X_tensor)):
            x_sample = X_tensor[i]
            y_sample = y_tensor[i]
            z = torch.dot(weight_tensor, x_sample) + bias_tensor
            y_pred = sigmoid(z)
            loss = compute_nll(y_pred, y_sample)
            cumulative_loss += loss.item()            
            error = y_pred - y_sample
            weight_gradient = error * x_sample
            bias_gradient = error            
            total_weight_gradient += weight_gradient
            total_bias_gradient += bias_gradient
        
        weight_gradient_mean = total_weight_gradient / len(X_tensor)
        bias_gradient_mean = total_bias_gradient / len(X_tensor)       
        weight_tensor -= learning_rate * weight_gradient_mean
        bias_tensor -= learning_rate * bias_gradient_mean
        avg_loss = cumulative_loss / len(X_tensor)
        loss_history.append(round(avg_loss, 4))
    
    final_weights = [round(w.item(), 4) for w in weight_tensor]
    final_bias = round(bias_tensor.item(), 4)   
    return final_weights, final_bias, loss_history

In [25]:
initial_weights = np.random.uniform(-1, 1, X_normalized.shape[1])
initial_bias = 0.0
learning_rate = 0.01
epochs = 20

weights, bias, loss_values = train_sigmoid_neuron(X_normalized, y_labels, initial_weights, initial_bias, learning_rate, epochs)

In [31]:
print("weights:", weights)
print()
print("bias:", bias)
print()
print("loss_values:", loss_values)

weights: [0.2386, 0.1989, -0.6175, 0.9336, -0.5482, 0.5449, 0.9188, -0.0322, -0.0354, 0.7702, -0.0661, -0.0636, 0.1232, -0.6076, 0.7494, -0.8349, 0.6, 0.5549, -0.3905, 0.9741, 0.6779, -0.0348, -0.9246, -0.595, -0.0519, 0.7853, -0.9505, -0.8597, -0.7947, -0.3133, 0.3675, 0.9624, 0.9575, 0.4339, 0.9713, -0.0362, 0.4268, -0.4664, -0.6041, 0.4399, -0.5329, 0.597, 0.3723, -0.9053, 0.3228, -0.1197, -0.1323, -0.2709, -0.023, 0.9897, 0.8285, -0.5294, -0.7971, 0.9163, -0.1953, 0.2388, 0.1027, -0.657, 0.887, 0.7506, 0.8712, 0.3836, 0.0872, 0.1045, 0.5627, -0.6615, -0.2895, -0.7575, 0.7575, 0.1646, 0.6435, -0.6168, 0.3126, -0.2266, 0.855, -0.4351, 0.6138, -0.034, 0.9445, -0.6722, -0.1426, 0.2638, -0.9277, -0.2057, -0.3085, -0.7369, -0.2361, -0.2306, 0.0858, 0.8779]

bias: 0.004

loss_values: [1.8611, 1.8594, 1.8576, 1.8559, 1.8541, 1.8524, 1.8507, 1.849, 1.8473, 1.8456, 1.8439, 1.8422, 1.8405, 1.8388, 1.8371, 1.8354, 1.8338, 1.8321, 1.8304, 1.8288]


In [33]:
predictions = sigmoid(torch.matmul(torch.tensor(X_normalized, dtype=torch.float64), torch.tensor(weights, dtype=torch.float64)) + bias)
rounded_predictions = torch.round(predictions)
accuracy = (rounded_predictions.numpy() == y_labels).mean()
print(f"accuracy: {round(accuracy * 100, 2)}%")

accuracy: 50.36%
